## Project Lending Club

In [71]:
from pyspark.sql import SparkSession
import getpass
username=getpass.getuser()

spark=SparkSession. \
builder. \
config('spark.shuffle.useOldFetchProtocol','true'). \
config("spark.sql.warehouse.dir",f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [10]:
spark

In [72]:
raw_df=spark.read. \
format("csv"). \
option("inferSchema","true"). \
option("header",'true'). \
load("/public/trendytech/datasets/accepted_2007_to_2018Q4.csv")

In [73]:
raw_df.createOrReplaceTempView("lendingclubdata")

In [74]:
spark.sql("select * from lendingclubdata limit 2")

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
14408468,null,5000.0,5000.0,5000.0,36 months,14.16,171.28,C,C2,CNC Machinist,8 years,MORTGAGE,50000.0,Not Verified,Apr-2014,Fully Paid,n,https://lendingcl...,null,debt_consolidation,Debt consolidation,571xx,SD,13.75,1.0,Sep-2004,675.0,679.0,1.0,4.0,107.0,8.0,1.0,2387.0,21.1,20.0,w,0.0,0.0,5561.49,5561.49,5000.0,561.49,0.0,0.0,0.0,Feb-2015,4019.97,null,Mar-2017,529.0,525.0,0.0,null,1.0,Individual,null,null,null,0.0,0.0,204115.0,null,null,null,null,null,null,null,null,null,null,null,11300.0,null,null,null,6.0,25514.0,1513.0,61.2,0.0,0.0,115.0,83.0,3.0,3.0,2.0,3.0,null,3.0,null,0.0,3.0,3.0,3.0,4.0,8.0,6.0,10.0,3.0,8.0,0.0,0.0,0.0,2.0,95.0,66.7,1.0,0.0,226297.0,19425.0,3900.0,18831.0,null,null,null,null,null,null,null,null,null,null,null,null,null,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Cash,N,null,null,null,null,null,null
14520567,null,11000.0,11000.0,11000.0,36 months,9.67,353.24,B,B1,null,null,OWN,34200.0,Verified,Apr-2014,Fully Paid,n,https://lendingcl...,null,other,Other,458xx,OH,28.63,0.0,Feb-2000,735.0,739.0,3.0,null,null,6.0,0.0,19566.0,95.4,20.0,f,0.0,0.0,12716.5251265996,12716.53,11000.0,1716.53,0.0,0.0,0.0,May-2017,353.13,null,Apr-2017,669.0,665.0,0.0,null,1.0,Individual,null,null,null,0.0,0.0,74047.0,null,null,null,null,null,null,null,null,null,null,null,20500.0,null,null,null,2.0,14809.0,null,null,0.0,0.0,116.0,170.0,21.0,2.0,1.0,112.0,null,2.0,null,0.0,0.0,3.0,1.0,6.0,3.0,4.0,16.0,3.0,6.0,0.0,0.0,0.0,1.0,100.0,null,0.0,0.0,93184.0,33050.0,0.0,13484.0,null,null,null,null,null,null,null,null,null,null,null,null,n

## Generating the member_id using sha2

In [75]:
from pyspark.sql.functions import sha2, concat_ws

### Adding a column

In [76]:
new_df=raw_df.withColumn("memberid_sha2",sha2(concat_ws("||",*["emp_title","emp_length","home_ownership","annual_inc","zip_code","addr_state","grade","sub_grade","verification_status"]),256))

In [77]:
new_df.createOrReplaceTempView("newtable")

In [78]:
spark.sql("select count(*) from newtable")

count(1)
2260701


In [79]:
spark.sql("select count(distinct(memberid_sha2)) from newtable")

count(DISTINCT memberid_sha2)
2257384


In [80]:
spark.sql("""
select memberid_sha2, count(*) as total_cnt
from newtable group by memberid_sha2 
having total_cnt>1
order by total_cnt desc
""").show()

+--------------------+---------+
|       memberid_sha2|total_cnt|
+--------------------+---------+
|e3b0c44298fc1c149...|       33|
|e4c167053d5418230...|        5|
|76b577467eda5bdbc...|        4|
|3f87585a20f702838...|        4|
|ad8e5d384dae17e06...|        4|
|22593a1870543b2db...|        3|
|e7d8d16928817ec8f...|        3|
|d9ce4046daf599732...|        3|
|c92062bb371842b3d...|        3|
|498bb6b1f0099cb47...|        3|
|5d52e7773cb0efff3...|        3|
|035bf3d8288d803bd...|        3|
|f54295a60946dedad...|        3|
|819453be77718d747...|        3|
|066ddaa64bee66dff...|        3|
|3ae415acd6bbfaac1...|        3|
|4ab6205de571ccb7b...|        3|
|c563428cb58da48ff...|        3|
|d3731bad3e6a82515...|        3|
|291ca1b09ef11ca3e...|        3|
+--------------------+---------+
only showing top 20 rows



In [81]:
spark.sql("select * from newtable where memberid_sha2 like 'e3b0c44298fc1c149%'")

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,memberid_sha2
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c44298fc1c149...
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e

In [82]:
spark.sql("""
select memberid_sha2 as member_id, emp_title, emp_length, home_ownership,
annual_inc, addr_state, zip_code, 'USA' as country, grade, sub_grade, 
verification_status, tot_hi_cred_lim, application_type, annual_inc_joint, verification_status_joint
from newtable
""")

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
6d5091b3fcaaeb4ea...,leadman,10+ years,MORTGAGE,55000.0,PA,190xx,USA,C,C4,Not Verified,178050.0,Individual,null,null
b5e7938b0a2da4cea...,Engineer,10+ years,MORTGAGE,65000.0,SD,577xx,USA,C,C1,Not Verified,314017.0,Individual,null,null
91060b858433e8a61...,truck driver,10+ years,MORTGAGE,63000.0,IL,605xx,USA,B,B4,Not Verified,218418.0,Joint App,71000.0,Not Verified
cab1fa9f533688b0a...,Information Syste...,10+ years,MORTGAGE,110000.0,NJ,076xx,USA,C,C5,Source Verified,381215.0,Individual,null,null
f74e401c1ab0adf78...,Contract Specialist,3 years,MORTGAGE,104433.0,PA,174xx,USA,F,F1,Source Verified,439570.0,Individual,null,null
8aef4bb29d609d8d6...,Veterinary Tecnician,4 years,RENT,34000.0,GA,300xx,USA,C,C3,Source Verified,16900.0,Individual,null,null
538b4653da3b1e814...,Vice President of...,10+ years,MORTGAGE,180000.0,MN,550xx,USA,B,B2,Not Verified,388852.0,Individual,null,null
b24d55f21390533c5...,road driver,10+ years,MORTGAGE,85000.0,SC,293xx,USA,B,B1,Not Verified,193390.0,Individual,null,null
1035c5401b0ca76d0...,SERVICE MANAGER,6 years,RENT,85000.0,PA,160xx,USA,A,A2,Not Verified,61099.0,Individual,null,null
cb0f1777593e77909...,Vendor liaison,10+ years,MORTGAGE,42000.0,RI,029xx,USA,B,B5,Not Verified,256513.0,Individual,null,null


In [83]:
spark.sql("""
select memberid_sha2 as member_id, emp_title, emp_length, home_ownership,
annual_inc, addr_state, zip_code, 'USA' as country, grade, sub_grade, 
verification_status, tot_hi_cred_lim, application_type, annual_inc_joint, verification_status_joint
from newtable
""").repartition(1).write \
.option("header","true") \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv007473/lendingclubproject/raw/customers_data_csv") \
.save()

In [84]:
customers_df=spark.read. \
format("csv"). \
option("inferSchema","true"). \
option("header",'true'). \
load("/user/itv007473/lendingclubproject/raw/customers_data_csv")

In [85]:
customers_df

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
4480925324607267c...,IT,3 years,RENT,92000.0,NJ,088xx,USA,E,E3,Verified,132523.0,Individual,null,null
b54711d4a553ea330...,District Manager,7 years,RENT,45000.0,TX,787xx,USA,C,C4,Verified,33247.0,Individual,null,null
db06b45a938f1a3b5...,Case Management,6 years,RENT,33000.0,NJ,070xx,USA,D,D3,Not Verified,34825.0,Individual,null,null
ad9d9524477e85c11...,senior financial ...,< 1 year,RENT,80000.0,NJ,079xx,USA,A,A4,Source Verified,81859.0,Individual,null,null
c67f6ac3fea6ef46d...,Senior Analyst H....,6 years,RENT,30000.0,FL,328xx,USA,B,B3,Source Verified,30825.0,Individual,null,null
bb36e2cb69517fac3...,Owner Realtor,1 year,MORTGAGE,40000.0,WI,549xx,USA,E,E3,Not Verified,186722.0,Individual,null,null
af69a7dff814fb213...,nurse,10+ years,MORTGAGE,45000.0,TX,791xx,USA,C,C5,Not Verified,261804.0,Individual,null,null
c9c794b5025e14a7d...,Assistant IT Dire...,2 years,MORTGAGE,76500.0,UT,840xx,USA,D,D5,Source Verified,67822.0,Individual,null,null
61b48d763bd82369a...,Network Provision...,3 years,RENT,98000.0,CA,950xx,USA,B,B2,Source Verified,100609.0,Individual,null,null
adc390ceaa6428ba4...,null,null,MORTGAGE,38964.0,MO,650xx,USA,C,C2,Source Verified,18400.0,Individual,null,null


In [86]:
spark.sql("""
select id as loan_id,memberid_sha2 as member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,
loan_status,purpose,title
from newtable
""").repartition(1).write \
.option("header","true") \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv007473/lendingclubproject/raw/loans_data_csv") \
.save()

In [87]:
loans_df=spark.read. \
format("csv"). \
option("inferSchema","true"). \
option("header",'true'). \
load("/user/itv007473/lendingclubproject/raw/loans_data_csv")

In [88]:
loans_df

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title
76003861,a4ec00ba67fadf2fe...,24000.0,24000.0,60 months,15.31,574.88,Apr-2016,Charged Off,debt_consolidation,Debt consolidation
76263914,4f7a9e6ffaacd5da2...,2400.0,2400.0,36 months,11.47,79.11,Apr-2016,Fully Paid,debt_consolidation,Debt consolidation
75537401,e935a4c27fc78ae61...,12600.0,12600.0,36 months,7.39,391.31,Apr-2016,Fully Paid,other,Other
75038986,2d32004bd5e1dc3c3...,16800.0,16800.0,60 months,19.53,440.72,Apr-2016,Current,credit_card,Credit card refin...
76301424,f7116b7f7546a7952...,4300.0,4300.0,36 months,17.27,153.89,Apr-2016,Charged Off,debt_consolidation,Debt consolidation
75333198,d3aa3a3c95eca5631...,8950.0,8950.0,36 months,22.45,343.9,Apr-2016,Current,credit_card,Credit card refin...
76391453,fc8a2e046eaaba02d...,35000.0,35000.0,60 months,12.99,796.18,Apr-2016,Fully Paid,debt_consolidation,Debt consolidation
76363364,577ae670ac2ec7ed3...,15000.0,15000.0,36 months,9.16,478.12,Apr-2016,Fully Paid,house,Home buying
76272510,d3792868819649ba9...,30000.0,30000.0,60 months,16.29,734.18,Apr-2016,Current,debt_consolidation,Debt consolidation
76304116,6d3a5a422261348b3...,4800.0,4800.0,36 months,19.99,178.37,Apr-2016,Fully Paid,credit_card,Credit card refin...


In [90]:
 spark.sql("""
select id as loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
from newtable
""").repartition(1).write \
.option("header","true") \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv007473/lendingclubproject/raw/loans_repayments_data_csv") \
.save()

In [91]:
loans_repayments_df=spark.read. \
format("csv"). \
option("inferSchema","true"). \
option("header",'true'). \
load("/user/itv007473/lendingclubproject/raw/loans_repayments_data_csv")

In [92]:
loans_repayments_df

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
56633077,3000.0,376.21,0.0,3376.205975527,93.74,Aug-2018,null
55927518,15600.0,1956.32,0.0,17556.320693408998,487.9,Aug-2018,null
56473345,20000.0,2408.94,0.0,22408.9398096902,9677.72,May-2017,null
56463188,11200.0,5231.01,0.0,16431.0146429476,7475.86,Feb-2018,null
56473316,5215.47,6513.51,0.0,13237.07,432.77,Nov-2017,null
56663266,6477.26,4221.77,0.0,12544.05,458.45,Jun-2017,null
56483027,10000.0,2062.03,0.0,12062.026276051001,335.38,Aug-2018,null
56613385,13932.74,11678.23,0.0,25610.97,609.46,Feb-2019,Apr-2019
56643620,16000.0,1031.67,0.0,17031.673055266598,8363.28,Mar-2017,null
56533114,3478.66,3815.07,0.0,12460.86,1111.52,Mar-2016,null


In [93]:
spark.sql("""
select memberid_sha2 as member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee, mths_since_last_delinq, mths_since_last_record
from newtable
""").repartition(1).write \
.option("header","true") \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv007473/lendingclubproject/raw/loans_defaulters_data_csv") \
.save()

In [94]:
loans_defaulters_df=spark.read. \
format("csv"). \
option("inferSchema","true"). \
option("header",'true'). \
load("/user/itv007473/lendingclubproject/raw/loans_defaulters_data_csv")

In [95]:
loans_defaulters_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
b59d80da191f5b573...,0.0,0.0,0.0,0.0,1.0,0.0,31.0,null
202d9f56ecb7c3bc9...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,null
e5a140c0922b554b9...,0.0,0.0,0.0,0.0,0.0,0.0,47.0,null
e12aefc548f750777...,0.0,0.0,0.0,0.0,0.0,0.0,33.0,null
1b3a50d854fbbf97e...,1.0,0.0,0.0,0.0,0.0,0.0,21.0,null
1c4329e5f17697127...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
5026c86ad983175eb...,0.0,0.0,1.0,0.0,2.0,0.0,null,71.0
9847d8c1e9d0b2084...,1.0,0.0,2.0,0.0,0.0,0.0,6.0,63.0
8340dbe1adea41fb4...,0.0,0.0,0.0,0.0,0.0,0.0,36.0,null
d4de0de3ab7d79ad4...,0.0,0.0,0.0,0.0,0.0,0.0,35.0,null
